In [15]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class PairedImageDataset(Dataset):
    def __init__(self, root_issba, root_train, transform=None):
        self.root_issba = root_issba
        self.root_train = root_train
        self.transform = transform

        # 获取train_issba目录下的所有文件路径
        self.issba_images = []
        for subdir, _, files in os.walk(root_issba):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png', '.JPEG', '.png')):
                    self.issba_images.append(os.path.join(subdir, file))

    def __len__(self):
        return len(self.issba_images)

    def __getitem__(self, idx):
        issba_image_path = self.issba_images[idx]
        image_name = os.path.basename(issba_image_path)  # 获取文件名

        # 获取对应的train文件夹下的图片路径
        train_image_path = issba_image_path.replace(self.root_issba, self.root_train)

        # 加载图像
        issba_image = Image.open(issba_image_path).convert("RGB")
        train_image = Image.open(train_image_path).convert("RGB")

        if self.transform:
            issba_image = self.transform(issba_image)
            train_image = self.transform(train_image)

        return issba_image, train_image

In [16]:
# import torch
# import numpy as np
# # from skimage.metrics import structural_similarity as ssim
# from pytorch_ssim import SSIM
# # from skimage.metrics import peak_signal_noise_ratio as psnr
# import lpips
# from torchmetrics.image import PeakSignalNoiseRatio
# import kornia.augmentation as A
# from optimize_filter.tiny_network import U_Net_tiny


# psnr = PeakSignalNoiseRatio()

# ssim = SSIM()

# # 创建LPIPS模型
# loss_fn = lpips.LPIPS(net='alex')

# def denormalize(tensor, mean, std):
#     """
#     反标准化函数：将标准化的张量恢复到原始像素值范围。
#     """
#     for t, m, s in zip(tensor, mean, std):
#         t.mul_(s).add_(m)
#     return tensor

# def calculate_metrics(dataloader, mean, std):
#     """
#     计算SSIM, PSNR和LPIPS指标

#     参数:
#     - dataloader: DataLoader - 数据加载器，提供img_backdoor和clean_img对
#     - mean: list - 标准化时使用的均值
#     - std: list - 标准化时使用的标准差

#     返回:
#     - avg_ssim: float - 平均SSIM
#     - avg_psnr: float - 平均PSNR
#     - avg_lpips: float - 平均LPIPS
#     """
#     ssim_scores = []
#     psnr_scores = []
#     lpips_scores = []

#     for img_backdoor, clean_img in dataloader:
#         # 反标准化图像
#         img_backdoor_denorm = denormalize(img_backdoor.clone(), mean, std)
#         clean_img_denorm = denormalize(clean_img.clone(), mean, std)

#         # 逐个样本计算SSIM和PSNR
#         # img_b = img_backdoor_denorm[i].numpy().transpose(1, 2, 0)  # (C, H, W) -> (H, W, C)
#         # img_c = clean_img_denorm[i].numpy().transpose(1, 2, 0)
#         img_b = img_backdoor_denorm
#         img_c = clean_img_denorm
#         # 计算SSIM，指定较小的win_size并设置channel_axis
#         ssim_value = ssim(img_b, img_c)
#         ssim_scores.append(ssim_value)

#         # 计算PSNR
#         psnr_value = psnr(img_b, img_c)
#         psnr_scores.append(psnr_value)

#         # 计算LPIPS
#         lpips_value = loss_fn(img_backdoor, clean_img)
#         lpips_scores.extend(lpips_value.detach().cpu().numpy())

#     # 计算平均值
#     avg_ssim = np.mean(ssim_scores)
#     avg_psnr = np.mean(psnr_scores)
#     avg_lpips = np.mean(lpips_scores)

#     return avg_ssim, avg_psnr, avg_lpips

In [17]:
import torch
import numpy as np
# from skimage.metrics import structural_similarity as ssim
from pytorch_ssim import SSIM
# from skimage.metrics import peak_signal_noise_ratio as psnr
import lpips
from torchmetrics.image import PeakSignalNoiseRatio
import kornia.augmentation as A
from optimize_filter.tiny_network import U_Net_tiny

# 检查CUDA设备

# 将模型和损失函数移动到CUDA
psnr = PeakSignalNoiseRatio()
ssim = SSIM()
loss_fn = lpips.LPIPS(net='alex')

def denormalize(tensor, mean, std):
    """
    反标准化函数：将标准化的张量恢复到原始像素值范围。
    """
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

def calculate_metrics(dataloader, mean, std, device):
    """
    计算SSIM, PSNR和LPIPS指标

    参数:
    - dataloader: DataLoader - 数据加载器，提供img_backdoor和clean_img对
    - mean: list - 标准化时使用的均值
    - std: list - 标准化时使用的标准差
    - device: torch.device - 计算设备（CPU或CUDA）

    返回:
    - avg_ssim: float - 平均SSIM
    - avg_psnr: float - 平均PSNR
    - avg_lpips: float - 平均LPIPS
    """
    ssim_scores = []
    psnr_scores = []
    lpips_scores = []

    for img_backdoor, clean_img in dataloader:
        # 移动到CUDA设备
        img_backdoor = img_backdoor
        clean_img = clean_img

        # 反标准化图像
        img_backdoor_denorm = denormalize(img_backdoor.clone(), mean, std)
        clean_img_denorm = denormalize(clean_img.clone(), mean, std)

        # 计算SSIM
        ssim_value = ssim(img_backdoor_denorm, clean_img_denorm)
        ssim_scores.append(ssim_value.item())

        # 计算PSNR
        psnr_value = psnr(img_backdoor_denorm, clean_img_denorm)
        psnr_scores.append(psnr_value.item())

        # 计算LPIPS
        lpips_value = loss_fn(img_backdoor_denorm, clean_img_denorm)
        lpips_scores.extend(lpips_value.detach().cpu().numpy())

    # 计算平均值
    avg_ssim = np.mean(ssim_scores)
    avg_psnr = np.mean(psnr_scores)
    avg_lpips = np.mean(lpips_scores)

    return avg_ssim, avg_psnr, avg_lpips



Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/jianzhang/anaconda3/envs/badencoder/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


# POIENC

In [22]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import lpips
import pandas as pd
from tqdm import tqdm
import torch

# 检查是否有可用的CUDA设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义输入和输出文件夹路径
input_folder = 'data/cifar10/test'
output_folder = 'data/cifar10/test_poienc'

# 初始化LPIPS模型并移动到GPU
lpips_model = lpips.LPIPS(net='alex').to(device)

# 创建字典来存储结果
results = {
    'filename': [],
    'ssim': [],
    'psnr': [],
    'lpips': []
}

# 遍历输入文件夹中的所有文件
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        # 获取原始图像和处理后图像的路径
        original_image_path = os.path.join(input_folder, filename)
        processed_image_path = os.path.join(output_folder, filename)

        # 检查处理后的图像是否存在
        if os.path.exists(processed_image_path):
            # 打开图像并转换为RGB图像以进行SSIM计算
            original_image = Image.open(original_image_path).convert('RGB')
            processed_image = Image.open(processed_image_path).convert('RGB')

            # 转换为NumPy数组
            original_array = np.array(original_image)
            processed_array = np.array(processed_image)

            # 计算SSIM，设置窗口大小为3，指定通道轴
            ssim_value = ssim(original_array, processed_array, win_size=3, channel_axis=-1)
            # 计算PSNR
            psnr_value = psnr(original_array, processed_array)

            # 计算LPIPS，转换图像为张量并移动到GPU
            original_tensor = lpips.im2tensor(original_array).to(device)
            processed_tensor = lpips.im2tensor(processed_array).to(device)
            lpips_value = lpips_model(original_tensor, processed_tensor)

            # 将结果存储到字典中
            results['filename'].append(filename)
            results['ssim'].append(ssim_value)
            results['psnr'].append(psnr_value)
            results['lpips'].append(lpips_value.item())
        else:
            print(f"处理后的图像未找到：{processed_image_path}")

# 将结果转换为DataFrame
df_results = pd.DataFrame(results)

# 计算平均值
average_ssim = df_results['ssim'].mean()
average_psnr = df_results['psnr'].mean()
average_lpips = df_results['lpips'].mean()

# 打印平均值
print(f"平均SSIM: {average_ssim}")
print(f"平均PSNR: {average_psnr}")
print(f"平均LPIPS: {average_lpips}")

# 保存结果到文件

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/jianzhang/anaconda3/envs/badencoder/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 10000/10000 [00:39<00:00, 253.53it/s]


平均SSIM: 0.12140157042403325
平均PSNR: 11.27871904912273
平均LPIPS: 0.15866825253739952


# BLTO

In [25]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import lpips
import pandas as pd
from tqdm import tqdm
import torch

# 检查是否有可用的CUDA设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义输入和输出文件夹路径
input_folder = 'data/cifar10/test'
output_folder = 'data/cifar10/BLTO_test'

# 初始化LPIPS模型并移动到GPU
lpips_model = lpips.LPIPS(net='alex').to(device)

# 创建字典来存储结果
results = {
    'filename': [],
    'ssim': [],
    'psnr': [],
    'lpips': []
}

# 遍历输入文件夹中的所有文件
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        # 获取原始图像和处理后图像的路径
        original_image_path = os.path.join(input_folder, filename)
        processed_image_path = os.path.join(output_folder, filename)

        # 检查处理后的图像是否存在
        if os.path.exists(processed_image_path):
            # 打开图像并转换为RGB图像以进行SSIM计算
            original_image = Image.open(original_image_path).convert('RGB')
            processed_image = Image.open(processed_image_path).convert('RGB')

            # 转换为NumPy数组
            original_array = np.array(original_image)
            processed_array = np.array(processed_image)

            # 计算SSIM，设置窗口大小为3，指定通道轴
            ssim_value = ssim(original_array, processed_array, win_size=3, channel_axis=-1)
            # 计算PSNR
            psnr_value = psnr(original_array, processed_array)

            # 计算LPIPS，转换图像为张量并移动到GPU
            original_tensor = lpips.im2tensor(original_array).to(device)
            processed_tensor = lpips.im2tensor(processed_array).to(device)
            lpips_value = lpips_model(original_tensor, processed_tensor)

            # 将结果存储到字典中
            results['filename'].append(filename)
            results['ssim'].append(ssim_value)
            results['psnr'].append(psnr_value)
            results['lpips'].append(lpips_value.item())
        else:
            print(f"处理后的图像未找到：{processed_image_path}")

# 将结果转换为DataFrame
df_results = pd.DataFrame(results)

# 计算平均值
average_ssim = df_results['ssim'].mean()
average_psnr = df_results['psnr'].mean()
average_lpips = df_results['lpips'].mean()

# 打印平均值
print(f"平均SSIM: {average_ssim}")
print(f"平均PSNR: {average_psnr}")
print(f"平均LPIPS: {average_lpips}")

# 保存结果到文件

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/jianzhang/anaconda3/envs/badencoder/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 10000/10000 [00:37<00:00, 263.23it/s]


平均SSIM: 0.8416590826636208
平均PSNR: 29.6755780377128
平均LPIPS: 0.009408540859771893


# SSLBKD

In [1]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import lpips
import pandas as pd
from tqdm import tqdm
import torch

# 检查是否有可用的CUDA设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义输入和输出文件夹路径
input_folder = 'data/cifar10/test'
output_folder = 'data/cifar10/test_SSLBKD'

# 初始化LPIPS模型并移动到GPU
lpips_model = lpips.LPIPS(net='alex').to(device)

# 创建字典来存储结果
results = {
    'filename': [],
    'ssim': [],
    'psnr': [],
    'lpips': []
}

# 遍历输入文件夹中的所有文件
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        # 获取原始图像和处理后图像的路径
        original_image_path = os.path.join(input_folder, filename)
        processed_image_path = os.path.join(output_folder, filename)

        # 检查处理后的图像是否存在
        if os.path.exists(processed_image_path):
            # 打开图像并转换为RGB图像以进行SSIM计算
            original_image = Image.open(original_image_path).convert('RGB')
            processed_image = Image.open(processed_image_path).convert('RGB')

            # 转换为NumPy数组
            original_array = np.array(original_image)
            processed_array = np.array(processed_image)

            # 计算SSIM，设置窗口大小为3，指定通道轴
            ssim_value = ssim(original_array, processed_array, win_size=3, channel_axis=-1)
            # 计算PSNR
            psnr_value = psnr(original_array, processed_array)

            # 计算LPIPS，转换图像为张量并移动到GPU
            original_tensor = lpips.im2tensor(original_array).to(device)
            processed_tensor = lpips.im2tensor(processed_array).to(device)
            lpips_value = lpips_model(original_tensor, processed_tensor)

            # 将结果存储到字典中
            results['filename'].append(filename)
            results['ssim'].append(ssim_value)
            results['psnr'].append(psnr_value)
            results['lpips'].append(lpips_value.item())
        else:
            print(f"处理后的图像未找到：{processed_image_path}")

# 将结果转换为DataFrame
df_results = pd.DataFrame(results)

# 计算平均值
average_ssim = df_results['ssim'].mean()
average_psnr = df_results['psnr'].mean()
average_lpips = df_results['lpips'].mean()

# 打印平均值
print(f"平均SSIM: {average_ssim}")
print(f"平均PSNR: {average_psnr}")
print(f"平均LPIPS: {average_lpips}")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/jianzhang/anaconda3/envs/badencoder/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jianzhang/anaconda3/envs/badencoder/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/jianzhang/anaconda3/envs/badencoder/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 10000/10000 [00:37<00:00, 266.72it/s]

平均SSIM: 0.8737167389974433
平均PSNR: 16.241425001927688
平均LPIPS: 0.09640422141216695


In [13]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import lpips
import pandas as pd
from tqdm import tqdm
import torch

# 检查是否有可用的CUDA设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义输入和输出文件夹路径
input_folder = 'data/imagenet/test'
output_folder = 'data/imagenet/test_badencoder_224'

# 初始化LPIPS模型并移动到GPU
lpips_model = lpips.LPIPS(net='alex').to(device)

# 创建字典来存储结果
results = {
    'filename': [],
    'ssim': [],
    'psnr': [],
    'lpips': []
}

# 定义目标尺寸
target_size = (224, 224)

# 遍历输入文件夹中的所有文件
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.JPEG'):
        # 获取原始图像和处理后图像的路径
        original_image_path = os.path.join(input_folder, filename)
        processed_image_path = os.path.join(output_folder, filename)

        # 检查处理后的图像是否存在
        if os.path.exists(processed_image_path):
            try:
                # 打开图像并转换为RGB图像
                original_image = Image.open(original_image_path).convert('RGB')
                processed_image = Image.open(processed_image_path).convert('RGB')

                # 调整图像大小为224x224
                original_image = original_image.resize(target_size, Image.LANCZOS)
                processed_image = processed_image.resize(target_size, Image.LANCZOS)

                # 转换为NumPy数组
                original_array = np.array(original_image)
                processed_array = np.array(processed_image)

                # 计算SSIM，设置窗口大小为3，指定通道轴
                ssim_value = ssim(original_array, processed_array, win_size=3, channel_axis=-1)

                # 计算PSNR
                psnr_value = psnr(original_array, processed_array)

                # 计算LPIPS，转换图像为张量并移动到GPU
                original_tensor = lpips.im2tensor(original_array).to(device)
                processed_tensor = lpips.im2tensor(processed_array).to(device)
                lpips_value = lpips_model(original_tensor, processed_tensor)

                # 将结果存储到字典中
                results['filename'].append(filename)
                results['ssim'].append(ssim_value)
                results['psnr'].append(psnr_value)
                results['lpips'].append(lpips_value.item())
            except Exception as e:
                print(f"处理图像 {filename} 时出错: {e}")
        else:
            print(f"处理后的图像未找到：{processed_image_path}")

# 将结果转换为DataFrame
df_results = pd.DataFrame(results)

# 计算平均值
average_ssim = df_results['ssim'].mean()
average_psnr = df_results['psnr'].replace([np.inf, -np.inf], np.nan).mean()
average_lpips = df_results['lpips'].mean()

# 打印平均值
print(f"平均SSIM: {average_ssim}")
print(f"平均PSNR: {average_psnr}")
print(f"平均LPIPS: {average_lpips}")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/jianzhang/anaconda3/envs/badencoder/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 100000/100000 [2:42:54<00:00, 10.23it/s]  


平均SSIM: 0.28003231653733107
平均PSNR: 9.984105166103491
平均LPIPS: 0.6105655954860896


In [10]:
average_ssim = df_results['ssim'].mean()
average_psnr=df_results['psnr'].replace([np.inf, -np.inf], np.nan).mean()
average_lpips = df_results['lpips'].mean()

# 打印平均值
print(f"平均SSIM: {average_ssim}")
print(f"平均PSNR: {average_psnr}")
print(f"平均LPIPS: {average_lpips}")

平均SSIM: 0.871243602042089
平均PSNR: 23.31519749462318
平均LPIPS: 0.05509908751847783


In [4]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import lpips
import pandas as pd
from tqdm import tqdm
import torch

# 检查是否有可用的CUDA设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义输入和输出文件夹路径
input_folder = 'data/imagenet/test'
output_folder = 'data/imagenet/test_badencoder'

# 初始化LPIPS模型并移动到GPU
lpips_model = lpips.LPIPS(net='alex').to(device)

# 创建字典来存储结果
results = {
    'filename': [],
    'ssim': [],
    'psnr': [],
    'lpips': []
}

# 遍历输入文件夹中的所有文件
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.JPEG'):
        # 获取原始图像和处理后图像的路径
        original_image_path = os.path.join(input_folder, filename)
        processed_image_path = os.path.join(output_folder, filename)

        # 检查处理后的图像是否存在
        if os.path.exists(processed_image_path):
            # 打开图像并转换为RGB图像以进行SSIM计算
            original_image = Image.open(original_image_path).convert('RGB')
            processed_image = Image.open(processed_image_path).convert('RGB')

            # 转换为NumPy数组
            original_array = np.array(original_image)
            processed_array = np.array(processed_image)

            # 计算SSIM，设置窗口大小为3，指定通道轴
            ssim_value = ssim(original_array, processed_array, win_size=3, channel_axis=-1)
            # 计算PSNR
            psnr_value = psnr(original_array, processed_array)

            # 计算LPIPS，转换图像为张量并移动到GPU
            original_tensor = lpips.im2tensor(original_array).to(device)
            processed_tensor = lpips.im2tensor(processed_array).to(device)
            lpips_value = lpips_model(original_tensor, processed_tensor)

            # 将结果存储到字典中
            results['filename'].append(filename)
            results['ssim'].append(ssim_value)
            results['psnr'].append(psnr_value)
            results['lpips'].append(lpips_value.item())
        else:
            print(f"处理后的图像未找到：{processed_image_path}")
        break

# 将结果转换为DataFrame
df_results = pd.DataFrame(results)

# 计算平均值
average_ssim = df_results['ssim'].mean()
average_psnr = df_results['psnr'].mean()
average_lpips = df_results['lpips'].mean()

# 打印平均值
print(f"平均SSIM: {average_ssim}")
print(f"平均PSNR: {average_psnr}")
print(f"平均LPIPS: {average_lpips}")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/jianzhang/anaconda3/envs/badencoder/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


  0%|          | 0/100000 [00:00<?, ?it/s]

平均SSIM: 0.7790360809085941
平均PSNR: 19.314246377824656
平均LPIPS: 0.05907715857028961
